# 2.7.Assistente de Controle de Despesas

### Importações

In [1]:
import csv
import sys
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

### Classe

In [2]:
class ExpenseAssistant:
    def __init__(self, training_file='expenses.csv'):
        self.expenses = []
        self.predicted_expenses = []
        self.vectorizer = CountVectorizer()
        self.classifier = MultinomialNB()
        self._train_classifier(training_file)
        self.load_expenses_from_file(training_file)

    def _train_classifier(self, training_file):
        training_data = []
        with open(training_file, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                description = row.get('description', '').strip()
                category = row.get('category', '').strip()
                if description and category:
                    training_data.append((description, category))
        texts, categories = zip(*training_data)
        X = self.vectorizer.fit_transform(texts)
        self.classifier.fit(X, categories)

    def predict_category(self, description):
        X_new = self.vectorizer.transform([description])
        prediction = self.classifier.predict(X_new)[0]
        return prediction

    def add_expense(self, amount, description, category=None):
        if not category or category.strip() == "":
            predicted_category = self.predict_category(description)
            self.predicted_expenses.append({"amount": amount, "description": description, "predicted_category": predicted_category})
            category = predicted_category
        self.expenses.append({"amount": amount, "description": description, "category": category})

    def load_expenses_from_file(self, file_path):
        with open(file_path, mode='r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                amount = float(row['amount'])
                description = row['description']
                category = row.get('category', '').strip()
                self.add_expense(amount, description, category)

    def summary(self):
        summary = {}
        for expense in self.expenses:
            summary.setdefault(expense["category"], 0)
            summary[expense["category"]] += expense["amount"]

        print("\nResumo de gastos:")
        for category, total in summary.items():
            print(f"{category}: R$ {total:.2f}")

        if self.predicted_expenses:
            print("\nDespesas cuja categoria foi prevista:")
            for expense in self.predicted_expenses:
                print(f"Descrição: {expense['description']} | Valor: R$ {expense['amount']:.2f} | Categoria prevista: {expense['predicted_category']}")


### Execução

In [3]:
if __name__ == "__main__":
    assistant = ExpenseAssistant("expenses.csv")
    assistant.summary()


Resumo de gastos:
Compras: R$ 19046.35
Transporte: R$ 13613.91
Alimentação: R$ 20266.54
Saúde: R$ 18189.74
Moradia: R$ 13099.73
Educação: R$ 16115.17
Outros: R$ 14652.03
Lazer: R$ 17787.06

Despesas cuja categoria foi prevista:
Descrição: Móveis | Valor: R$ 10.10 | Categoria prevista: Compras
Descrição: Mercado | Valor: R$ 439.75 | Categoria prevista: Alimentação
Descrição: Internet | Valor: R$ 336.66 | Categoria prevista: Moradia
Descrição: Mercado | Valor: R$ 225.61 | Categoria prevista: Alimentação
Descrição: Gasolina | Valor: R$ 360.36 | Categoria prevista: Transporte
Descrição: Padaria | Valor: R$ 52.94 | Categoria prevista: Alimentação
Descrição: Óculos | Valor: R$ 466.30 | Categoria prevista: Saúde
